In [2]:
def get_clone_cell_embed(adata_obj, clone_table):
    '''
    Creates clone cell embedding, returns combined object with new dims and cluster assignments
    clone_table: columns: first column: cell.bc, second column: clone_id, (opt) third column and beyond: metadata to add to final object
    '''
    
    all_clones = clone_table['clone_id'].unique()
    all_obs = pd.concat((pd.Series(adata_obj.obs_names),pd.Series(all_clones)))
    
    #create coembed object
    adata_obj_coembed = sc.AnnData(np.zeros(len(all_obs),100))
    
    #create new connectivities based on edge list
    adata_obj_conn = adata_obj.obsp['connectivities']
    
    #impute clone cell edges
    imp_rows = all_clones
    imp_cols = adata_obj.obs_names
    
    #clone x cell connectivites
    conn_impute = np.zeros((len(all_clones), len(adata_obj.obs_names)))
    for i in clone_table.iterrows():
        row_idx = i[1][1]
        col_idx = i[1][0]

        row_idx = np.where(imp_rows == row_idx)[0][0]
        col_idx = np.where(imp_cols == col_idx)[0][0]
        conn_impute[row_idx, col_idx] = 1
        
    adata_obj_conn2 = scipy.sparse.vstack((adata_obj_conn, scipy.sparse.csr_matrix(conn_impute)))
    adata_obj_conn3 = scipy.sparse.vstack((scipy.sparse.csr_matrix(conn_impute).transpose(),
                                        scipy.sparse.csr_matrix(np.zeros((len(all_clones),len(all_clones))))))
    
    adata_obj_conn_coembed = scipy.sparse.hstack((adata_obj_conn2, adata_obj_conn3))
    adata_obj_conn_coembed = adata_obj_conn_coembed.tocsr()
    
    #assert that mtx is symmetric and diagonal is zero
    assert((adata_obj_conn_coembed - adata_obj_conn_coembed.T).sum() == 0)
    assert((adata_obj_conn_coembed.diagonal()==0).all())
    
    
    #add data to coembed object
    adata_obj_coembed.uns['neighbors'] = dict()
    adata_obj_coembed.uns['neighbors']['params'] = adata_obj_coembed.uns['neighbors']['params']
    adata_obj_coembed.obsp['connectivities'] = adata_obj_conn_coembed
    
    #run draw graph
    sc.tl.draw_graph(adata_obj_coembed)
    
    #run leiden
    sc.tl.leiden(adata_obj_coembed, key_added='leiden_coemb')
    
    #add metadata to clone object, if any
    clone_table.index = clone_table.clone_id
    adata_obj_coembed.obs = adata_obj_coembed.obs.merge(clone_table.iloc[:,1:].drop_duplicates(keep='first', subset = 'clone_id'),
                                   left_index=True, right_index=True,validate='one_to_one', how='left')
    
    return(adata_obj_coembed)

## Write a refined clone coembed function

In [3]:
import numpy as np
import pandas as pd
import scanpy as sc

In [ ]:
adata_cell = sc.read_h5ad("../pro")